In [ ]:
#!/usr/bin/env python
import torch
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import Video

from core.tools import initialize, visualize

torch.multiprocessing.set_sharing_strategy("file_system")

In [ ]:
cfg, model, dataset, epic_classes, device = initialize("config/config_vis.yaml")
data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=24,
        shuffle=False,
        num_workers=8,
    )

In [ ]:
%matplotlib inline
interact(visualize, cfg=fixed(cfg), model=fixed(model), dataset=fixed(dataset), index=widgets.IntSlider(min=1, max=len(dataset), step=1, value=0, continuous_update=False), epic_classes=fixed(epic_classes), device=fixed(device))

In [ ]:
Video("results/temp.MP4", height=256, width=455)